In [1]:
!pip install transformers datasets torch

   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   ----- ---------------------------------- 1.3/10.0 MB 6.7 MB/s eta 0:00:02
   ---------- ----------------------------- 2.6/10.0 MB 6.6 MB/s eta 0:00:02
   ---------------- ----------------------- 4.2/10.0 MB 6.6 MB/s eta 0:00:01
   --------------------- ------------------ 5.5/10.0 MB 6.6 MB/s eta 0:00:01
   --------------------------- ------------ 6.8/10.0 MB 6.5 MB/s eta 0:00:01
   ------------------------------ --------- 7.6/10.0 MB 6.5 MB/s eta 0:00:01
   ------------------------------------- -- 9.4/10.0 MB 6.5 MB/s eta 0:00:01
   ---------------------------------------- 10.0/10.0 MB 6.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/203.0 MB ? eta -:--:--
   ---------------------------------------- 1.3/203.0 MB 6.1 MB/s eta 0:00:34
    --------------------------------------- 2.6/203.0 MB 6.6 MB/s eta 0:00:31
    --------------------------------------- 3.9/203.0 MB 6.5 MB/s eta 0:00:31
   - ---

In [3]:
!pip install tf-keras

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------------------ --------------- 1.0/1.7 MB 6.3 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 6.2 MB/s eta 0:00:00


In [13]:
!pip install transformers[torch] accelerate

In [14]:
pip show transformers accelerate torch

Name: transformers
Version: 4.46.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: c:\Users\Admin\.conda\envs\supply_chain\Lib\site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 
---
Name: accelerate
Version: 1.1.1
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: c:\Users\Admin\.conda\envs\supply_chain\Lib\site-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 
---
Name: torch
Version: 2.5.1
Summary: Tensors and Dynamic neu

ERROR: Invalid requirement: "'accelerate": Expected package name at the start of dependency specifier
    'accelerate
    ^


In [1]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

c:\Users\Admin\.conda\envs\supply_chain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load dataset
df = pd.read_csv(r"data\data_trustpilot.csv") 
df.head()

,rating,location,username,number_reviews,verification,repeat_reviewer,repeat_reviewer_encoded,company,text,text_processed,...,date_posted,local_date_posted,month_local,local_hour,time_of_day,day_of_week_posted,day_type,days_between_experience_and_post,review_time,review_time_encoded
0,5,CA,Rob Crane,2,Redirected,repeat,1,Flashbay,The company rep I worked with made my transact...,company rep worked made transaction smooth qui...,...,2024-10-23 04:17:44,2024-10-22,10,21,Evening,1,Business Day,129,late_review,0
1,5,US,Pat Anderson,1,Verified,one-time,0,Flashbay,I highly recommend using Flashbay. Immediately...,highly recommend using flashbay immediately or...,...,2024-10-16 19:34:05,2024-10-16,10,12,Business Hours,2,Business Day,0,quick_review,1
2,5,CZ,Margarita Orlova,1,Verified,one-time,0,Flashbay,I had the pleasure of working with Shelby Gibs...,pleasure working shelby gibson large order nee...,...,2024-10-17 10:27:44,2024-10-17,10,10,Business Hours,3,Business Day,7,late_review,0
3,5,US,Paola Rivas,1,Verified,one-time,0,Flashbay,I had a fantastic experience with Brian Truong...,fantastic experience brian truong attentive tr...,...,2024-10-21 22:38:50,2024-10-21,10,15,Business Hours,0,Business Day,0,quick_review,1
4,5,CA,Fiona Mckelvey Keenan,3,Not Verified,repeat,1,Flashbay,My number-one go-to for computer accessories. ...,numberone goto computer accessories rachel sup...,...,2024-10-23 04:09:05,2024-10-22,10,21,Evening,1,Business Day,103,late_review,0


In [3]:
# readjustment to make it fit in the model 
df['rating'] = df['rating'] - 1

In [ ]:
# Select the column to use as input 
input_column = 'text_processed'  

# Tokenization and Dataset Preparation
class ReviewDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        text = row[input_column]
        rating = row['rating']
        tokens = self.tokenizer(
            text, 
            max_length=self.max_length, 
            truncation=True, 
            padding='max_length', 
            return_tensors='pt'
        )
        return {
            'input_ids': tokens['input_ids'].squeeze(),
            'attention_mask': tokens['attention_mask'].squeeze(),
            'labels': torch.tensor(rating, dtype=torch.long)
        }

# Initialize tokenizer and model
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=5)

# Prepare Dataset
max_length = 128
dataset = ReviewDataset(df, tokenizer, max_length)

# Split dataset into train and test
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = ReviewDataset(train_df, tokenizer, max_length)
test_dataset = ReviewDataset(test_df, tokenizer, max_length)

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"
)

# Initialize Trainer
def compute_metrics(eval_pred):
    from sklearn.metrics import accuracy_score, precision_recall_fscore_support
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=1)
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(f"Test Results: {results}")



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Admin\AppData\Local\Temp\ipykernel_9796\3313736649.py:72: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
                                        
  0%|          | 0/9663 [04:05<?, ?it/s]            

{'loss': 1.5118, 'grad_norm': 6.097047328948975, 'learning_rate': 1.997930249404947e-05, 'epoch': 0.0}


                                        
  0%|          | 0/9663 [07:17<?, ?it/s]            

{'loss': 1.3196, 'grad_norm': 4.611196994781494, 'learning_rate': 1.9958604988098936e-05, 'epoch': 0.01}


                                        
  0%|          | 0/9663 [10:26<?, ?it/s]            

...

{'loss': 0.776, 'grad_norm': 4.619821071624756, 'learning_rate': 1.9751629928593606e-05, 'epoch': 0.04}


                                        
  0%|          | 0/9663 [41:56<?, ?it/s]             

{'loss': 0.8392, 'grad_norm': 8.294193267822266, 'learning_rate': 1.9730932422643073e-05, 'epoch': 0.04}


                                        
  0%|          | 0/9663 [45:30<?, ?it/s]             

{'loss': 0.7373, 'grad_norm': 12.684285163879395, 'learning_rate': 1.971023491669254e-05, 'epoch': 0.04}
